In [5]:
def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]


import re

def replace_multiple_newlines(text):
    # 3つ以上の連続する改行を2つの改行に置換
    return re.sub(r'\n{2,}', '\n\n', remove_spaces_from_blank_lines(text))

def remove_spaces_from_blank_lines(text):
    # 各行を処理し、スペースのみの行を空行に置き換える
    return re.sub(r'^\s+$', '', text, flags=re.MULTILINE)
def select_usable_xmm(usable_var):
    count_true = sum(usable_var)
    count_mod2 = (count_true>>1)<<1
    count_half = count_true>>1
    indices = find_true_indices(usable_var, count_mod2)
    
    vars1 = [f"xmm{idx}" for idx in indices[:count_half]]
    vars2 = [f"xmm{idx}" for idx in indices[count_half:]]
    for idx in indices:
        usable_var[idx] = False
    return vars1, vars2, usable_var 

def find_true_indices(lst, N):
    true_indices = [i for i, val in enumerate(lst) if val]
    return true_indices[:N]

def select_usable_var(usable_var, prefix="r", set_second=False):
    var1, var2 = "", ""
    indices = find_true_indices(usable_var, 2)
    
    var1 = f"{prefix}{indices[0]}"
    var2 = f"{prefix}{indices[1]}"
    usable_var[indices[0]] = False
    if set_second: usable_var[indices[1]] = False
    return var1, var2, usable_var

def accumulator_naive_c(accm_op, variables, indent_level=1):
    indent = " " * 4 * indent_level
    
    accm_op.append("\n")
    if len(variables) == 1:
        accm_op.append(variables[0])
        return 

    variables_next_ = []
    variables_ = variables[:-1] if len(variables) % 2 == 1 else variables
    for i in range(0, len(variables_), 2):
        accm_op.append(f"{indent}{variables_[i]} = _mm_add_pd({variables_[i]}, {variables_[i+1]});")
        variables_next_.append(f"{variables_[i]}")
        
    if len(variables) % 2 == 1:
        variables_next_.append(f"{variables[-1]}")
        
    accumulator_naive_c(accm_op, variables_next_, indent_level = indent_level)


with open(f"./small_dgemv_naive_c_implementation_ver1_5.c", "w") as p:
    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("""
double sum_xmm_elements_v1_5(__m128d xmmX) {
    __m128d xmm_high_low = _mm_add_pd(xmmX, _mm_unpackhi_pd(xmmX, xmmX));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}    
    """)

    for ldx in range(2, 514, 2):
        ldx_max = min([ldx, 14])
        declare_xmm = "__m128d " + ", ".join([f"xmm{ld}" for ld in range(ldx_max)])
        variables = [f"xmm{ld}" for ld in range(ldx_max)]

        usable_var = [True] * ldx_max
        
        vars1, vars2, usable_var = select_usable_xmm(usable_var)
    
        load_xmm = ""
        op_concat = ""
        
        if ldx<=1000:
            if ldx <=15:
                for i, var in enumerate(vars2):
                    load_xmm += f"""
        {var} = _mm_loadu_pd(&x[{i*2}]);"""


                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm_loadu_pd(&a_t[{i*2}]);"""

                op_concat += "\n"
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])

                op_concat += f"\n        y[i] = sum_xmm_elements_v1_5({accm_op[-1]}); a_t += {ldx};"

            else:
                for i, var in enumerate(vars2):
                    op_concat += f"""
            {var} = _mm_loadu_pd(&x[{i*2}]);"""
                for i, var in enumerate(vars1):
                    op_concat += f"""
            {var} = _mm_loadu_pd(&a_t[{i*2}]);"""
                    
                for var1, var2 in zip(vars1, vars2):
                    op_concat += f"""
            {var1} = _mm_mul_pd({var1}, {var2});"""

                accm_variables = vars1
                accm_op = []
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                for var1, var2 in zip(vars1, vars2):
                    usable_var[int(var1[3:])] = True
                    usable_var[int(var2[3:])] = True
                usable_var[0] = False

                for i in range(14, ldx, 2):
                    var1, var2, usable_var = select_usable_var(usable_var, prefix="xmm", set_second=False)

                    op_concat += f"""
            {var1} = _mm_loadu_pd(&x[{i}]);
            {var2} = _mm_loadu_pd(&a_t[{i}]);
            {var1} = _mm_mul_pd({var1}, {var2});
                    """
                    if sum(usable_var)<=1:
                        accm_op = []
                        accm_variables = [f"xmm{i}" for i in range(len(usable_var)) if not usable_var[i]]

                        accumulator_naive_c(accm_op, accm_variables, indent_level=4)
                        op_concat += "\n".join(accm_op[:-1])
                        for var in accm_variables[1:]:
                            idx = int(var[3:])
                            usable_var[idx] = True        
                accm_op = []
                accm_variables = [f"xmm{i}" for i in range(len(usable_var)) if not usable_var[i]]
                accumulator_naive_c(accm_op, accm_variables, indent_level=2)
                op_concat += "\n".join(accm_op[:-1])
                op_concat += f"\n        y[i] = sum_xmm_elements_v1_5({accm_op[-1]}); a_t += {ldx};"
            
        base = f"""
void mydgemv_t_ver1_5_{ldx}(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    {declare_xmm};
    {load_xmm}
    
    for (int64_t i=0; i<lda; i++){{
        {op_concat}
        
    }}
}}
        """
        
        p.write(base)

In [5]:
for i in range(2, 514, 2):
    print(f"""
        subroutine mydgemv_t_ver1_5_{i}(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_t_ver1_5_{i}')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_t_ver1_5_{i}
          """)



        subroutine mydgemv_t_ver1_5_2(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_t_ver1_5_2')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_t_ver1_5_2
          

        subroutine mydgemv_t_ver1_5_4(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_t_ver1_5_4')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_t_ver1_5_4
          

        subroutine mydgemv_t_ver1_5_6(a_t, x, y, lda, ldx, ldy, shift) bind(C, name='mydgemv_t_ver1_5_6')
            import
            integer(c_int64_t), value :: lda, ldx, ldy, shift
            real(c_double), intent(in) :: a_t(ldx, lda), x(ldx), y(ldy)
        end subroutine mydgemv_t_ver1_5_6
          

        subroutine mydgemv_t_ver1_5_8(a_t, x, y, lda, ldx, l